In [1]:
# !pip install pandas requests

In [2]:
# !pip install pdfminer.six

In [3]:
# !pip install beautifulsoup4

In [ ]:
# !pip install tqdm

Objetivo:

Resgatar dados que auxiliem no acompanhamento das informações dos eventos do legislativo.


Como recuperar algo que seja do interesse de determinado tema?
Como recuperar algo que seja de interesse de outro órgão ou tipo de interessado?


Facilita a organização e o processo de informação frente a quantia de eventos na agenda, possibilitando gerar alertas caso algo que esteja presente em pauta impacte os interessados. Posteriormente é possível analisar o quanto determinada pauta ou assunto interfira e seja benéfico para população (de forma subjetiva usando LLMs).

In [4]:
import pandas as pd
import requests
from datetime import datetime, timedelta

from pdfminer.high_level import extract_text
from bs4 import BeautifulSoup

from urllib.parse import urlparse, parse_qs
import os


Checar se o dia da semana é final de semana, de forma a condicionar de pegar a agenda dessa semana ou da próxima.

In [5]:
today = datetime.today()
weekday = today.weekday()

if weekday < 5:
    start_date = today

    days_until_saturday = 5 - today.weekday() # Sábado é valor 5 
    end_date = today + timedelta(days=days_until_saturday)

elif weekday == 5:
    start_date = today + timedelta(days=1)
    end_date = today + timedelta(days=7)

else:
    start_date = today
    end_date = today + timedelta(days=6)


print(f"Data de início: {start_date.strftime('%Y-%m-%d')}")
print(f"Data de fim: {end_date.strftime('%Y-%m-%d')}")


Data de início: 2025-06-29
Data de fim: 2025-07-05


In [6]:
def get_events(start_date, end_date, itens_per_page=100):
    url = "https://dadosabertos.camara.leg.br/api/v2/eventos"
    headers = {
        "Accept": "application/json"
    }

    page = 1
    events = []
    while True:
        params = {
            "dataInicio": start_date,
            "dataFim": end_date,
            "ordenarPor": "dataHoraInicio",
            "itens": itens_per_page,
            "ordem": "ASC",
            "pagina": page
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code != 200:
            print(f"Erro ao requisitar página {page}: {response.status_code}")
            break

        data = response.json()["dados"]

        if not data:
            break

        events.extend(data)
        print(f"Página {page} carregada com {len(data)} eventos.")
        page += 1

    return events

In [7]:
events = get_events(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'))

Página 1 carregada com 80 eventos.


In [8]:
print(f"Total de eventos encontrados: {len(events)}")

Total de eventos encontrados: 80


In [9]:
events_df = pd.DataFrame(events)
events_df.head()

,id,uri,dataHoraInicio,dataHoraFim,situacao,descricaoTipo,descricao,localExterno,orgaos,localCamara,urlRegistro
0,75916,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-01T10:00,None,Cancelada,Audiência Pública,Adaptação das Estruturas Físicas das Escolas F...,Plenário a definir,"[{'id': 2009, 'uri': 'https://dadosabertos.cam...","{'nome': None, 'predio': None, 'sala': None, '...",None
1,76160,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-01T13:00,None,Convocada,Audiência Pública,Planejamento e Diretrizes das ações do Governo...,None,"[{'id': 537480, 'uri': 'https://dadosabertos.c...","{'nome': 'Anexo II, Plenário 13', 'predio': No...",None
2,76192,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-01T13:00,None,Convocada,Seminário,Inclusão de diretrizes para uma educação antir...,None,"[{'id': 539384, 'uri': 'https://dadosabertos.c...","{'nome': 'Anexo II, Plenário 12', 'predio': No...",None
3,76323,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-02T11:00,None,Agendada,Sessão Não Deliberativa Solene,"Homenagem aos 77 anos da Nakba, a catástrofe p...",None,"[{'id': 180, 'uri': 'https://dadosabertos.cama...","{'nome': 'Plenário da Câmara dos Deputados', '...",None
4,76326,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-02T09:00,None,Agendada,Sessão Não Deliberativa Solene,Homenagem aos 20 anos do Instituto Sabin\r\n S...,None,"[{'id': 180, 'uri': 'https://dadosabertos.cama...","{'nome': 'Plenário da Câmara dos Deputados', '...",None


Temos que um evento pode estar associado a várias comissões, elas estão dispostas na coluna órgãos. Pode ser interessante realizar um repasse dos dataframes para SQL de forma que possamos pegar informações estatísticas atreladas também as comissões. Não podemos garantir que sempre terá apenas uma comissão associada.

In [10]:
event_sample = events_df.iloc[0]
event_sample.orgaos

[{'id': 2009,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009',
  'sigla': 'CE',
  'nome': 'Comissão de Educação',
  'apelido': 'Educação',
  'codTipoOrgao': 2,
  'tipoOrgao': 'Comissão Permanente',
  'nomePublicacao': 'Comissão de Educação',
  'nomeResumido': 'Educação'}]

Analisando a URL disposta nos dados, obtemos um XML com informações adicionais e links interessantes.

In [11]:
event_sample.uri

'https://dadosabertos.camara.leg.br/api/v2/eventos/75916'

In [12]:
event_detail_url = event_sample.uri

response = requests.get(event_detail_url, headers={"Accept": "application/json"})

if response.status_code == 200:
    event_detail = response.json()
    print(f"Detalhes resgatados: {event_detail}")
else:
    print(f"Erro na requisição de detalhes {response.status_code}")


Detalhes resgatados: {'dados': {'uriDeputados': None, 'uriConvidados': None, 'fases': None, 'id': 75916, 'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/75916', 'dataHoraInicio': '2025-07-01T10:00', 'dataHoraFim': None, 'situacao': 'Cancelada', 'descricaoTipo': 'Audiência Pública', 'descricao': 'Adaptação das Estruturas Físicas das Escolas Frente às Mudanças Climáticas\r\n 1)   (a Confirmar) Representante do  Ministério da Educação (MEC);\r\n\r\n2) (a Confirmar)  Representante do  Ministério do Meio Ambiente e Mudança do Clima; \r\n\r\n3)  (a Confirmar)  Representante do Conselho Nacional de Secretários de Educação; \r\n\r\n4)  (a Confirmar) Representante da União Nacional dos Dirigentes Municipais de Educação (Undime); \r\n\r\n5) CAYO ALCÂNTARA  (Confirmado) - Professor e Diretor-Executivo de ‘A Vida No Cerrado’; \r\n\r\n6) HAROLDO ROCHA (Confirmado) - Representante do Instituto Península e do Movimento Profissão Docente ;\r\n\r\n7) (a Confirmar) Representante do Instituto A

In [13]:
event_detail

{'dados': {'uriDeputados': None,
  'uriConvidados': None,
  'fases': None,
  'id': 75916,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/75916',
  'dataHoraInicio': '2025-07-01T10:00',
  'dataHoraFim': None,
  'situacao': 'Cancelada',
  'descricaoTipo': 'Audiência Pública',
  'descricao': 'Adaptação das Estruturas Físicas das Escolas Frente às Mudanças Climáticas\r\n 1)   (a Confirmar) Representante do  Ministério da Educação (MEC);\r\n\r\n2) (a Confirmar)  Representante do  Ministério do Meio Ambiente e Mudança do Clima; \r\n\r\n3)  (a Confirmar)  Representante do Conselho Nacional de Secretários de Educação; \r\n\r\n4)  (a Confirmar) Representante da União Nacional dos Dirigentes Municipais de Educação (Undime); \r\n\r\n5) CAYO ALCÂNTARA  (Confirmado) - Professor e Diretor-Executivo de ‘A Vida No Cerrado’; \r\n\r\n6) HAROLDO ROCHA (Confirmado) - Representante do Instituto Península e do Movimento Profissão Docente ;\r\n\r\n7) (a Confirmar) Representante do Instituto Alan

A partir daqui temos alguns caminhos interessantes para explorar, como:

- Requerimentos, temos o link do requerimento associado ao evento
- Pautas: Podemos ter um sumário diferente ao que o evento se refere, nas pautas também podemos encontrar os convidades e representantes necessários para o evento.

A informação de convidados pelo que é visto ao acessar os documentos é repetida em ambos, porém alguns eventos não possuem requerimento, apenas pauta.

Essa resposta de detalhes está defasada em alguns pontos:
- Temos que links leva a si mesmo
- 'urlDocumentoPauta' também não é válido e leva a um XML simples que não possui tanta informação. Devido a isso devemos achar outra forma de encontrar a pauta. Pensando nisso adentrei em uma pauta manual pelo site da agenda, podemos realizar um Webscrap com a url base, alterando apenas o código da pauta.

In [14]:
event_id = event_detail["dados"]["id"]
event_url = f"https://www.camara.leg.br/evento-legislativo/{event_id}"
event_url

'https://www.camara.leg.br/evento-legislativo/75916'

Verificando o site da câmara para extrair a pauta temos:
(inserir imagem)


Com isso podemos utilizar o BeautifulSoup para extrair o link necessário de acesso a Pauta.

In [15]:
# Necessário para webscrap para que o requests aja como um navegador
headers = {
    "User-Agent": "Mozilla/5.0"
}

response_event = requests.get(event_url, headers=headers)

if response_event.status_code == 200:
    soup_event = BeautifulSoup(response_event.text, "html.parser")
    itens = soup_event.find_all(class_="links-adicionais__item")

    cleaned_links = {}
    for item in itens:
        texto = item.get_text(strip=True)
        link = item.find("a")["href"] if item.find("a") else None

        cleaned_links[texto] = link
        
        print(f"Texto: {texto}")
        print(f"Link: {link}")
else:
    print(f"Erro ao acessar a página: {response.status_code}")

Texto: Requerimento
Link: https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2869237
Texto: Documentos da reunião
Link: #documentos-modal
Texto: Pauta
Link: https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2923746


Aqui temos por exemplo o link também para o requerimento.

In [16]:
cleaned_links

{'Requerimento': 'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2869237',
 'Documentos da reunião': '#documentos-modal',
 'Pauta': 'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2923746'}

In [17]:
# Necessário para webscrap para que o requests aja como um navegador
headers = {
    "User-Agent": "Mozilla/5.0"
}

response_topics = requests.get(cleaned_links['Pauta'], headers=headers)
if response_topics.status_code != 200:
    raise Exception(f"Erro {response_topics.status_code}")

soup_topics = BeautifulSoup(response_topics.text, "html.parser")


In [18]:
soup_topics

<!--
   Licensed to the Apache Software Foundation (ASF) under one or more
   contributor license agreements.  See the NOTICE file distributed with
   this work for additional information regarding copyright ownership.
   The ASF licenses this file to You under the Apache License, Version 2.0
   (the "License"); you may not use this file except in compliance with
   the License.  You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
-->
<!-- prolog breaks internet explorer, WICKET-2718 -->
<!-- < ? xml version="1.0" encoding="UTF-8" ? > -->
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/lo

Obtemos que a URL disponível no site é inválida, na realidade ela mostra que é feito um redirecionamento automático para outra URL! Podemos utilizar esta outra URL novamente como uma URL base e alterar apenas o código que direciona para a pauta. URL que é direcionado: https://www.camara.leg.br/internet/ordemdodia/integras/

In [19]:
# Resgatar o ID que estava como query parameter na URL antiga
parsed_url = urlparse(cleaned_links['Pauta'])
query_params = parse_qs(parsed_url.query)

codteor = query_params.get("codteor", [None])[0]

true_topics_url = f"https://www.camara.leg.br/internet/ordemdodia/integras/{codteor}.htm"
true_topics_url

'https://www.camara.leg.br/internet/ordemdodia/integras/2923746.htm'

In [20]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(true_topics_url, headers=headers)

if response.status_code != 200:
    raise Exception(f"Erro {response.status_code}")

soup_true_topics = BeautifulSoup(response.text, "html.parser")


Neste ponto temos que a Pauta possui uma diagramação de HTML muito bagunçada e cheia de divisões! Também não poderia garantir que todas as pautas possuiriam as mesmas áreas e tópicos de texto. Devido a isso decidi resgatar todo o texto como um grande bloco, possível posteriormente pegar "a partir de tal palavra", por hora dessa forma é suficiente.

In [21]:
# Extrai o texto visível da página, sem tags HTML
topics_text = soup_true_topics.get_text(separator="\n", strip=True)

print(topics_text[:2000])

Pauta - CE - 01/07/2025 10:00
CÂMARA DOS DEPUTADOS
COMISSÃO DE EDUCAÇÃO
57ª Legislatura - 3ª Sessão Legislativa Ordinária
PAUTA DE
REUNIÃO EXTRAORDINÁRIA
AUDIÊNCIA PÚBLICA
DIA 01/07/2025
LOCAL:
Plenário a definir
HORÁRIO:
10h
TEMA:
"Adaptação das Estruturas Físicas das Escolas Frente às Mudanças Climáticas"
1)  (a Confirmar) Representante do Ministério da Educação (MEC);
2) (a Confirmar) Representante do Ministério do Meio Ambiente e Mudança do Clima;
3) (a Confirmar) Representante do Conselho Nacional de Secretários de Educação;
4) (a Confirmar) Representante da União Nacional dos Dirigentes Municipais de Educação (Undime);
5) CAYO ALCÂNTARA (Confirmado) - Professor e Diretor-Executivo de A Vida No Cerrado;
6) HAROLDO ROCHA (Confirmado) - Representante do Instituto Península e do Movimento Profissão Docente ;
7) (a Confirmar) Representante do Instituto Alana;
(REQ 2/2025 CE, da deputada Socorro Neri)


---

Partindo para os requerimentos, temos que nas informações de detalhes do evento há URIs funcionais para acessá-los, a url direciona para um XML de detalhes e neste XML temos o caminho para o documento inteiro. Vamos resgatá-lo para salvarmos todo o texto e fazermos análise textual. Há a possibilidade de criarmos outro DataFrame com informações apenas dos requerimentos, uma vez que um evento pode ter vários.

In [22]:
motion_url = event_detail['dados']['requerimentos'][0]['uri']

response_motion = requests.get(motion_url, headers={"Accept": "application/json"})

if response_motion.status_code == 200:
    print(response_motion.json())

motion_detail = response_motion.json()

motion_detail

{'dados': {'id': 2487727, 'uri': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2487727', 'siglaTipo': 'REQ', 'codTipo': 294, 'numero': 2, 'ano': 2025, 'ementa': 'Requer a realização de Audiência Pública para discutir como as escolas estão adaptando suas estruturas físicas às mudanças climáticas.', 'dataApresentacao': '2025-03-19T12:38', 'uriOrgaoNumerador': None, 'statusProposicao': {'dataHora': '2025-03-26T10:00', 'sequencia': 2, 'siglaOrgao': 'CE', 'uriOrgao': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009', 'uriUltimoRelator': None, 'regime': '.', 'descricaoTramitacao': 'Aprovação de Proposição Interna', 'codTipoTramitacao': '241', 'descricaoSituacao': 'Aguardando Providências Internas', 'codSituacao': 936, 'despacho': 'Aprovado o Requerimento, com subscrição das Deputadas Tabata Amaral e Carol Dartora e do Deputado Tarcísio Motta.', 'url': None, 'ambito': 'Regimental', 'apreciacao': 'Indefinida'}, 'uriAutores': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes

{'dados': {'id': 2487727,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2487727',
  'siglaTipo': 'REQ',
  'codTipo': 294,
  'numero': 2,
  'ano': 2025,
  'ementa': 'Requer a realização de Audiência Pública para discutir como as escolas estão adaptando suas estruturas físicas às mudanças climáticas.',
  'dataApresentacao': '2025-03-19T12:38',
  'uriOrgaoNumerador': None,
  'statusProposicao': {'dataHora': '2025-03-26T10:00',
   'sequencia': 2,
   'siglaOrgao': 'CE',
   'uriOrgao': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009',
   'uriUltimoRelator': None,
   'regime': '.',
   'descricaoTramitacao': 'Aprovação de Proposição Interna',
   'codTipoTramitacao': '241',
   'descricaoSituacao': 'Aguardando Providências Internas',
   'codSituacao': 936,
   'despacho': 'Aprovado o Requerimento, com subscrição das Deputadas Tabata Amaral e Carol Dartora e do Deputado Tarcísio Motta.',
   'url': None,
   'ambito': 'Regimental',
   'apreciacao': 'Indefinida'},
  'uriAutor

Do requerimento temos de interessante:  Ementa, Autores e URL de inteiro Teor.

Até então temos a estrutura:
- Evento
  - Pode possuir vários requerimentos
    - Um requerimento pode possuir vários autores.
    - Possui uma pauta

Diferente das pautas que estão em HTML os documentos são lidos como PDF, ou seja, precisamos baixá-los e depois convertemos para um texto tratável.

In [26]:
import requests

motions_folder = '../data/motions/'

motion_pdf_url = motion_detail['dados']['urlInteiroTeor']

headers = {
    "User-Agent": "Mozilla/5.0",  # finge ser um navegador
    "Accept": "application/pdf"
}

response_motion_pdf = requests.get(motion_pdf_url, headers=headers)

motion_number = motion_detail['dados']['numero']
motion_year = motion_detail['dados']['ano']

pdf_name = os.path.join(motions_folder, f"{motion_number}_{motion_year}.pdf")

if response_motion_pdf.status_code == 200:
    with open(pdf_name, "wb") as f:
        f.write(response_motion_pdf.content)
    print("PDF baixado com sucesso.")
else:
    print(f"Erro {response_motion_pdf.status_code}")


PDF baixado com sucesso.


Vamos testar extrair o texto de um único documento.

In [27]:
texto = extract_text(pdf_name)

# Exibe os primeiros caracteres
print(texto[:1000])

CÂMARA DOS DEPUTADOS
Gabinete da Deputada Socorro Neri PP/AC

COMISSÃO DE EDUCAÇÃO

REQUERIMENTO Nº       , DE 2025
(Da Sra. Socorro Neri)

Requer   a   realização   de   Audiência   Pública
para   discutir   como   as   escolas   estão
adaptando   suas   estruturas   físicas   às
mudanças climáticas.

Senhor Presidente, 

Requeiro   a   Vossa   Excelência,   nos   termos   do   art.   24,   inciso   III   do
Regimento   Interno   da   Câmara   dos   Deputados,   a   realização   de   Audiência
Pública com a finalidade de debater como as instituições de ensino estão se
adaptando às mudanças climáticas, investigando a existência e adequação de
protocolos   específicos   para   a   garantia   do   conforto   térmico   e   bem-estar   de
alunos, professores e demais funcionários.

  Proponho   para   participar   desta   Audiência   Pública   os   seguintes

convidados:

1. Representante do Ministério da Educação (MEC);

2. Representante do    Ministério do Meio Ambiente e Mudança do Clim

Com isso temos uma estrutura de resgate dos dados que desejamos! Para melhor Organizar a criação de dataframes, vamos repassar os códigos aqui desenvolvidos para scripts mais funcionais e organizados. Presentes na pasta `src`.